In [1]:
import os
import sys
import time

dir = os.path.abspath('')
while not dir.endswith('ardt'): dir = os.path.dirname(dir)
if not dir in sys.path: sys.path.append(dir)

In [2]:
import io
import time

import paramiko

from utils.helpers import find_root_dir

from private_keys import GATEWAY_PASSWORD, MYRIAD_PRIVATE_KEY, GITHUB_PRIVATE_KEY

In [3]:
HOME_DIR = '/home/ucabscm'
ARDT_DIR = find_root_dir()

## Connecting to Myriad

In [ ]:
# Private key
k = MYRIAD_PRIVATE_KEY.strip()
padding = len(k) % 4
if padding > 0:
    k += "=" * (4 - padding)
private_key_file = io.StringIO(k)
pkey = paramiko.RSAKey.from_private_key(private_key_file)

# Establish SSH connection to the first server (gateway)
sshgat = paramiko.SSHClient()
sshgat.set_missing_host_key_policy(paramiko.AutoAddPolicy())
sshgat.connect('ssh-gateway.ucl.ac.uk', username='ucabscm', password=GATEWAY_PASSWORD)

# Use this connection to establish a second SSH connection to the second server (myriad)
# Paramiko Transport is used here for nested ssh
transport = sshgat.get_transport()
dest_addr = ('myriad.rc.ucl.ac.uk', 22) # IP and port
local_addr = ('ssh-gateway.ucl.ac.uk', 22) # IP and port
jump = transport.open_channel("direct-tcpip", dest_addr, local_addr)

myriad = paramiko.SSHClient()
myriad.set_missing_host_key_policy(paramiko.AutoAddPolicy())
myriad.connect('myriad.rc.ucl.ac.uk', username='ucabscm', sock=jump, pkey=pkey)

## Set up environment, code, files

In [ ]:
def sftp_upload_dir(sftp, localdir, remotedir):
    os.chdir(os.path.split(localdir)[0])
    parent = os.path.split(localdir)[1]
    for walker in os.walk(parent):
        try:
            sftp.mkdir(os.path.join(remotedir, walker[0]))
        except Exception as e:
            print("Exception:", e)
        for file in walker[2]:
            sftp.put(os.path.join(walker[0], file), os.path.join(remotedir, walker[0], file))

In [ ]:
stdin, stdout, stderr = myriad.exec_command(f"""
                                            module purge &&
                                            module load default-modules &&
                                            module remove mpi compilers &&
                                            module load python/3.9.10
                                            """.strip().replace("\n", " "))
stdout.channel.recv_exit_status()

In [ ]:
stdin, stdout, stderr = myriad.exec_command(f"""
                                            rm -rf {HOME_DIR}/action-robust-decision-transformer &&
                                            git clone https://{GITHUB_PRIVATE_KEY}@github.com/afonsosamarques/action-robust-decision-transformer.git
                                             """.strip().replace("\n", " "))
stdout.channel.recv_exit_status()

In [ ]:
# # NOTE: not always needed...
# stdin, stdout, stderr = myriad.exec_command(f"""
#                                             module load python/3.9.10 &&
#                                             mkdir {HOME_DIR}/envs &&
#                                             python3 -m venv {HOME_DIR}/envs/ardt-env &&
#                                             source {HOME_DIR}/envs/ardt-env/bin/activate &&
#                                             pip install --upgrade pip &&
#                                             pip install -r {HOME_DIR}/action-robust-decision-transformer/requirements.txt
#                                              """.strip().replace("\n", " "))
# stdout.channel.recv_exit_status()

In [ ]:
stdin, stdout, stderr = myriad.exec_command(f"""
                                            mkdir {HOME_DIR}/action-robust-decision-transformer/codebase/ardt/eval-outputs &&
                                            mkdir {HOME_DIR}/action-robust-decision-transformer/codebase/ardt/eval-outputs-pipeline &&
                                            mkdir {HOME_DIR}/action-robust-decision-transformer/codebase/ardt/eval-outputs-test &&
                                            mkdir {HOME_DIR}/action-robust-decision-transformer/codebase/ardt/agents &&
                                            mkdir {HOME_DIR}/action-robust-decision-transformer/codebase/ardt/agents-pipeline &&
                                            mkdir {HOME_DIR}/action-robust-decision-transformer/codebase/ardt/agents-test &&
                                            mkdir {HOME_DIR}/action-robust-decision-transformer/codebase/ardt/wandb &&
                                            mkdir {HOME_DIR}/action-robust-decision-transformer/codebase/ardt/wandb-json
                                            """.strip().replace("\n", " "))
stdout.channel.recv_exit_status()

In [ ]:
sftp = myriad.open_sftp()

sftp.put(f'{ARDT_DIR}/access_tokens.py', f"{HOME_DIR}/action-robust-decision-transformer/codebase/ardt/access_tokens.py")
time.sleep(2)

In [ ]:
# # NOTE: not always needed...
# sftp_upload_dir(sftp, f'{ARDT_DIR}/datasets', f"{HOME_DIR}")
# time.sleep(2)

In [ ]:
stdin, stdout, stderr = myriad.exec_command(f'cp -r {HOME_DIR}/datasets {HOME_DIR}/action-robust-decision-transformer/codebase/ardt') 
stdout.channel.recv_exit_status()

## Run script and close connections

In [ ]:
stdin, stdout, stderr = myriad.exec_command(f"""
                                            qsub {HOME_DIR}/action-robust-decision-transformer/codebase/ardt/all-notebooks/clusters-nbs/myriad-run-scripts/run_experiment_7.sh &&
                                            qsub {HOME_DIR}/action-robust-decision-transformer/codebase/ardt/all-notebooks/clusters-nbs/myriad-run-scripts/run_experiment_8.sh
                                            """.strip().replace("\n", " "))
stdout.channel.recv_exit_status()

In [ ]:
time.sleep(10)
sftp.close()
myriad.close()
sshgat.close()